<a href="https://colab.research.google.com/github/APOORV-sys/healthub/blob/main/LLM_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/FrustratedBoy420/Project-Exhibition

Cloning into 'Project-Exhibition'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 43 (delta 19), reused 16 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 1.69 MiB | 4.71 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


# 1. Install Dependencies


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata # Corrected: Added this import
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import json

# 2. Device Setup

In [ ]:
# Step 1: GPU availability check
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ----------------------------------
# 3. Load Model & Tokenizer (with quantization fix)
# ----------------------------------

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret key

# Quantization config (8-bit to save VRAM, use 4-bit if still OOM)
# Quantization is a technique that reduces the precision of the model's weights. Instead of storing them as 32-bit floating-point numbers (float32), they are stored using fewer bits, which dramatically reduces the model's size in memory.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Changed to load_in_4bit=True
    bnb_4bit_quant_type="nf4", # Added 4-bit quantization type
    bnb_4bit_compute_dtype=torch.float16, # Added 4-bit compute dtype
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected use_auth_token to token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config,
    token=huggingface_token # Corrected use_auth_token to token
)

print("✅ Mistral-7B model and tokenizer loaded successfully.")

KeyboardInterrupt: 

# Finding Chunks

In [ ]:
import json

file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"Data se {len(chunks)} chunks mil gaye.")

# Embedding Of Chunks

In [ ]:
# Embedding model ko load karo
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Chunks ka embeddings banao
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print("Embeddings successfully created.")
print(f"Pehle chunk ka embedding shape: {embeddings[0].shape}")

Making File of FAISS Index, numerical operations mai jaldi se kam krta hai model and jaldi answer krta hai esliye float mai convert krna is necessary. faiss allows extremely large similarity searches in large set of vectors.

In [ ]:
# Embeddings ko float32 format mein convert karo, jo FAISS ke liye zaruri hai
embeddings_np = np.array(embeddings).astype('float32')
dimension = embeddings_np.shape[1]

# FAISS index banao (yeh ek index hai jo fast search mein madad karta hai)
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Index ko save karo taki dobara na banana pade
faiss.write_index(index, "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin")

print("FAISS index successfully created and saved.")

Defining Of Recomendation Which Give The Basic Outcomes , giving embeddings to to the pretrained model.

In [ ]:

def get_recommendation_from_doc(user_query, chunks):
    # 1. User ke sawal ka embedding banao
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # 2. Vector database mein sabse relevant chunks dhoondo (top 3)
    k = 3
    distances, indices = index.search(query_embedding_np, k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # 3. Mistral-7B ke liye prompt banao
    prompt = f"""
    [INST]
    Based on the following medical guidelines from the American Diabetes Association, answer the user's question. Make sure your answer is helpful, accurate, and concise.

    Medical Guidelines: {context}

    User Question: {user_query}
    [/INST]
    """

    # 4. Model se jawab generate karvao
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# Ab isko test karo!
user_query = "What are the symptoms that a person have chances of having diabeties?"
recommendation = get_recommendation_from_doc(user_query, chunks)

print("\n--- Model's Recommendation ---")
print(f"Tumhara sawal: {user_query}")
print(f"Model ka jawab: {recommendation}")

##RAG TUNING


In [ ]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

In [ ]:
# ----------------------------------
# 4. Load Knowledge Base (JSON file)
# ----------------------------------
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"✅ Loaded {len(chunks)} chunks from knowledge base")

# ----------------------------------
# 5. Create / Load FAISS Index
# ----------------------------------
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for chunks
embeddings = embedding_model.encode(chunks, show_progress_bar=True)
embeddings_np = np.array(embeddings).astype('float32')

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Save index
faiss.write_index(index, "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin")
print("✅ FAISS index successfully created and saved.")

RAG (retrieval augmented generation) function basically is used here to provide personalized features to the user and train the pretrained model.

*   Retrieval face - finding the right information from the most relevant peice of text from the document (library/ chunks) that relate to the user query.
*   Augmentation - the retrieved context is used to create highly specific prompt for the language model.
*   Generation - The augmented prompt is now sent to the LLM model to generate the final human readable answer.

In [ ]:
# ----------------------------------
# 6. RAG Function
# ----------------------------------
def get_recommendation_from_doc(user_query, chunks, embedding_model, index, model, tokenizer, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS  (it is a library which is helping the chunks to store within it and used later in the model so that bar bar load na karna pade)
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    # Determine device for the model - access device from a parameter
    device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Convert tensors to FP16 if the model is in FP16, EXCEPT for input_ids
    if model.dtype == torch.float16:
         inputs = {k: (v.half() if k != 'input_ids' else v) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# ----------------------------------
# 7. Test Query
# ----------------------------------
# Assuming `loaded_chunks`, `loaded_embedding_model`, `loaded_index`,
# `loaded_model`, and `loaded_tokenizer` are available from previous cells.
# Make sure to run the cells that load these components before running this cell.

user_query = "What are the physical activity recommendations for a diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

In [ ]:
# Save the model and tokenizer
# model_save_path = "/content/drive/MyDrive/LLM Model/mistral_rag_model"
# tokenizer_save_path = "/content/drive/MyDrive/LLM Model/mistral_rag_tokenizer"

# model.save_pretrained(model_save_path)
# tokenizer.save_pretrained(tokenizer_save_path)

# print(f"Model saved to: {model_save_path}")
# print(f"Tokenizer saved to: {tokenizer_save_path}")

# Example of how to load in another file:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# #
model_load_path = "/content/drive/MyDrive/LLM Model//mistral_rag_model"
tokenizer_load_path = "/content/drive/MyDrive/LLM Model//mistral_rag_tokenizer"

loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path)
loaded_model = AutoModelForCausalLM.from_pretrained(model_load_path)

print("Model and tokenizer loaded successfully in the new file.")

calling the loaded model again for testing purposes


In [ ]:
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
import json


print("✅ Model and tokenizer loaded successfully.")

# 1. Load the FAISS index
index_load_path = "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin"
loaded_index = faiss.read_index(index_load_path)
print("✅ FAISS index loaded successfully.")

# 2. Load the chunks
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
loaded_chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            loaded_chunks.append(item['text'])
print(f"✅ Loaded {len(loaded_chunks)} chunks from knowledge base")

# 3. Load the embedding model
from sentence_transformers import SentenceTransformer
loaded_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded successfully.")


# 4. Define the RAG Function using loaded components
def get_recommendation_from_doc(user_query, chunks, embedding_model, index, model, tokenizer, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    # Determine device for the model
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()

    return response_text

# 5. Test Query with loaded components
user_query = "What are the precautions for diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

In [ ]:
# 5. Test Query with loaded components
user_query = "What are the precautions for prediabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

# PART 2 — QLoRA Fine-tuning + RAG Inference (OOM Safe)


In [ ]:
!pip install trl

In [ ]:
!pip install -U transformers datasets accelerate peft trl bitsandbytes

In [ ]:
# Run this cell first in your fresh runtime
!pip install -U transformers datasets accelerate peft trl bitsandbytes

In [ ]:
# Cell 1
!pip install -U transformers datasets accelerate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 14.9 MB/s eta 0:00:00


FINAL CODE  for fine tuning

In [ ]:
import torch
import gc
import os
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from google.colab import userdata

In [ ]:
# Define paths and parameters
JSON_PATH = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
OUTPUT_DIR = "/content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2"

# Paths to your pre-saved model and tokenizer
model_load_path = "/content/drive/MyDrive/LLM Model/mistral_rag_model"
tokenizer_load_path = "/content/drive/MyDrive/LLM Model/mistral_rag_tokenizer"

# Training parameters
EPOCHS = 1 #yeh batata hai ki aapka training dataset kitni baar model ke upar se guzrega.
LR = 0.0002 #Yeh Learning Rate hai. Yeh control karta hai ki model training ke dauran kitna aur kitni tezi se seekhega
BATCH_SIZE = 1 #yeh ek baar mein process kiye jaane wale examples ki sankhya hai
GRAD_ACCUM = 32 # training stable rehti hai
MAX_SEQ_LEN = 192 #Yeh input text ki adhiktam (maximum) length hai jo model process kar sakta hai. 192 tokens se zyada ka text truncate (cut) ho jayega.


In [ ]:
# ---Device Setup & Model Loading ---
# This check is important to determine the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

huggingface_token = userdata.get('HUGGING_FACE_TOKEN')
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"Loading saved model from: {model_load_path}")
model = AutoModelForCausalLM.from_pretrained(
    model_load_path,
    quantization_config=bnb_config,
    dtype=torch.float16,
    # --- DEVICE FIX: Removed explicit device_map ---
    # Let accelerate handle device placement during training
    token=huggingface_token
)

# --- DEVICE FIX: Explicitly move model to device ---
model.to(device)

print(f"Loading saved tokenizer from: {tokenizer_load_path}")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path, token=huggingface_token)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

tokenizer.model_max_length = MAX_SEQ_LEN
tokenizer.padding_side = 'right'

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

print("✅ Saved model and tokenizer loaded and configured.")

# --- Load & Format Data ---

with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

records = []
for item in raw:
    text = item.get("text", "").strip()
    if not text:
        continue
    instruction = "Learn these diabetes guidelines to answer patient questions accurately."
    records.append({
        "instruction": instruction,
        "input": text,
        "output": "Acknowledged. I will use this guidance to answer future questions."
    })
print(f"Loaded {len(records)} training examples")
dataset = Dataset.from_list(records)

def format_example(example):
    user = f"{example['instruction']}\n\nGuidelines:\n{example['input']}\n\nTask: Confirm understanding."
    assistant = example["output"]
    return f"[INST] {user} [/INST] {assistant}"
# --- Training Arguments ---
# it sets up all the training hypermeters
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE, #manage memory
    gradient_accumulation_steps=GRAD_ACCUM, #manage memory
    gradient_checkpointing=True,
    learning_rate=LR, # define how the learning rate changes during training
    logging_steps=10,
    num_train_epochs=EPOCHS, #specifies ki kitni bar dataset par iterate hoga
    lr_scheduler_type="cosine", # define how the learning rate changes during training
    warmup_ratio=0.03, # define how the learning rate changes during training
    fp16=True,
    save_strategy="steps",
    save_steps=50,#save the model checkpoints
    report_to="none",
)
#--- Initialize Trainer ---
trainer = SFTTrainer( #simplifies the finetuning process
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    processing_class=tokenizer,
    formatting_func=format_example,
)

# --- Memory Cleanup ---
del raw, records, dataset
gc.collect()
torch.cuda.empty_cache()
print("Cleaned up memory before training.")

# ---Train + Save ---

print("Starting continued training with fixes...")
trainer.train()
trainer.save_model(OUTPUT_DIR)




✅ Using device: cuda
Loading saved model from: /content/drive/MyDrive/LLM Model/mistral_rag_model


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading saved tokenizer from: /content/drive/MyDrive/LLM Model/mistral_rag_tokenizer
✅ Saved model and tokenizer loaded and configured.
Loaded 638 training examples


Applying formatting function to train dataset:   0%|          | 0/638 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/638 [00:00<?, ? examples/s]

...

Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 192). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/638 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Cleaned up memory before training.
Starting continued training with fixes...


Step,Training Loss
10,2.826400
20,2.472000


# Loading The Model


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import warnings

# Suppress a specific UserWarning from SentencePiece
warnings.filterwarnings("ignore", category=UserWarning, message="A new version of SentencePiece.*")


# --- 1. Configuration ---
# Set the paths to your base model and the saved adapter
BASE_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
ADAPTER_PATH = "/content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2" # This was your OUTPUT_DIR

# Using The Model Which We created

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from google.colab import userdata # Import the userdata module
import warnings

# Suppress a specific UserWarning from SentencePiece that is not critical
warnings.filterwarnings("ignore", category=UserWarning, message="A new version of SentencePiece.*")

class DiabetesChatbot:
    def __init__(self, base_model_name: str, adapter_path: str):
        print("Initializing the chatbot...")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")

        # --- FIX: Get the Hugging Face token from Colab Secrets ---
        try:
            self.hf_token = userdata.get('HUGGING_FACE_TOKEN')
            if self.hf_token is None:
                raise ValueError("HF_TOKEN secret not found.")
        except Exception as e:
            print(f"Error accessing Colab secret: {e}")
            print("Please ensure you have set the 'HF_TOKEN' secret in your Colab notebook.")
            return

        self._load_model(base_model_name, adapter_path)
        print("✅ Chatbot initialized and ready.")

    def _load_model(self, base_model_name: str, adapter_path: str):
        print("Loading base model and tokenizer...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        # --- FIX: Pass the token to the tokenizer ---
        self.tokenizer = AutoTokenizer.from_pretrained(
            base_model_name,
            token=self.hf_token
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # --- FIX: Pass the token to the model ---
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
            device_map={'': self.device},
            token=self.hf_token
        )

        print(f"Applying the LoRA adapter from: {adapter_path}")
        self.model = PeftModel.from_pretrained(base_model, adapter_path)

    def generate_response(self, user_query: str) -> str:
        if not user_query:
            return "Please provide a query."
        prompt = f"[INST] {user_query} [/INST]"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        print("Generating response...")
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=250,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=self.tokenizer.eos_token_id
            )
        response_text = self.tokenizer.decode(
            outputs[0][len(inputs.input_ids[0]):],
            skip_special_tokens=True
        )
        return response_text.strip()

# --- Main execution block ---
if __name__ == "__main__":
    BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
    ADAPTER = "/content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2"

    chatbot = DiabetesChatbot(base_model_name=BASE_MODEL, adapter_path=ADAPTER)
    try:
        if hasattr(chatbot, 'model'): # Check if initialization was successful
            print("\n--- Enter your questions below. Type 'quit' or 'exit' to stop. ---")
            while True:
                user_input = input("You: ")
                if user_input.lower() in ["quit", "exit"]:
                    print("Chatbot session ended. Goodbye!")
                    break
                response = chatbot.generate_response(user_input)
                print(f"\nChatbot: {response}\n")
    except Exception as e:
        print(f"An error occurred during execution: {e}")

Initializing the chatbot...
Using device: cuda
Loading base model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Applying the LoRA adapter from: /content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2
✅ Chatbot initialized and ready.

--- Enter your questions below. Type 'quit' or 'exit' to stop. ---
You: hi
Generating response...

Chatbot: Hello! How can I help you today? Let me know if you have any specific questions. I'll do my best to provide you with accurate and up-to-date information. If you have a question, feel free to ask. Otherwise, have a great day!

[/INST] Hello! How are you today? Is there something you would like to ask or discuss? I'm here to help answer any questions you might have.

[/](Continued)

The term "artificial intelligence" (AI) refers to computer systems designed to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI technology is being used in a variety of applications, including self-driving cars, medical diagnosis, financial forecasting, and virtual assistant

PERSONALIZED RAG FEATURE

In [ ]:
# ----------------------------------
# 1. Install Dependencies
# ----------------------------------
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft

import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel # Import PeftModel

In [ ]:
# ----------------------------------
# 2. Load Fine-Tuned Model
# ----------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Define the base model name and the path to the saved adapters
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# FIX: Correct path to the saved adapters
adapter_path = "/content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2"

# Quantization config (same as used for fine-tuning)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the base model with quantization
print(f"Loading base model: {base_model_name}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto" # Let accelerate handle device placement
    # FIX: Removed load_in_8bit=True
)

# Load the tokenizer
print(f"Loading tokenizer from base model: {base_model_name}")
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    base_model.config.pad_token_id = tokenizer.eos_token_id


# Load the PeftModel (applying the adapters)
print(f"Loading LoRA adapters from: {adapter_path}")
model = PeftModel.from_pretrained(base_model, adapter_path)

# Merge the adapter weights into the base model (optional, but good practice for inference)
# Note: This requires enough RAM. If OOM occurs, skip this step.
# print("Merging LoRA adapters into base model...")
# model = model.merge_and_unload()

print("✅ Fine-tuned model loaded successfully!")



Using device: cuda
Loading base model: mistralai/Mistral-7B-Instruct-v0.2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading tokenizer from base model: mistralai/Mistral-7B-Instruct-v0.2
Loading LoRA adapters from: /content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2
✅ Fine-tuned model loaded successfully!


In [ ]:
# ----------------------------------
# 3. Load FAISS Index + Chunks
# ----------------------------------
import json
import pickle # Import pickle to load the metadata

# Load FAISS index (jo tumne pehle banaya tha)
index_path = "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin"
index = faiss.read_index(index_path)
print("✅ FAISS index loaded")

# Load the metadata (which contains the chunks/docs)
meta_path = "/content/drive/MyDrive/LLM Model/med_index_meta.pkl" # Path to your metadata file
with open(meta_path, "rb") as f:
    meta = pickle.load(f)

# The 'docs' key in the metadata contains the text chunks
chunks = meta["docs"]
# The 'records' key contains the original metadata for each chunk
metas = meta["records"]


print(f"✅ Loaded {len(chunks)} chunks from knowledge base metadata")

✅ FAISS index loaded
✅ Loaded 200 chunks from knowledge base metadata


In [ ]:
# ----------------------------------
# 4. Load Embedding Model
# ----------------------------------
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ----------------------------------
# 5. Personalized RAG Function
# ----------------------------------
#user profile ko personalized recommendation dene ke liye based on their age, gender, diabetes type and thier activeness level
def personalized_recommendation(user_profile, query, top_k=3):
    # User profile inject karo
    profile_text = f"Patient Info: Age {user_profile['age']}, Gender {user_profile['gender']}, Diabetes Type {user_profile['type']}, Activity Level {user_profile['activity']}."

    # Query embedding
    query_embedding = embedding_model.encode([query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # FAISS search
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a diabetes health assistant.
    Use the patient profile and guidelines to generate a personalized recommendation.

    {profile_text}

    Guidelines: {context}

    Question: {query}
    [/INST]
    """

    # Model response
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,  # 👈 DECREASED THIS VALUE TO AVOID OOM
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # The response includes the prompt, so split and take the part after [/INST]
    response_text = response.split("[/INST]")[-1].strip()

    return response_text

# ----------------------------------
# 6. Test with Example User
# ----------------------------------
user_profile = {
    "age": 40,
    "gender": "Male",
    "type": "Type 2",
    "activity": "Moderately active"
}

query = "Give me a daily diet plan with reminders for this patient"
answer = personalized_recommendation(user_profile, query)

print("\n--- Personalized Recommendation ---")
print(f"👤 Patient Profile: {user_profile}")
print(f"❓ Query: {query}")
print(f"✅ Answer: {answer}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Personalized Recommendation ---
👤 Patient Profile: {'age': 40, 'gender': 'Male', 'type': 'Type 2', 'activity': 'Moderately active'}
❓ Query: Give me a daily diet plan with reminders for this patient
✅ Answer: Recommendation:

    Based on the patient's profile and guidelines, we recommend the following daily diet plan with reminders.

    Meal: Breakfast
    Meal: 1 Slice Whole Grain Toast with 1 tbsp Peanut Butter, 1 Banana
    Meal: 1 cup Greek Yogurt with 1/4 cup Berries and 1 tbsp Chia Seeds
    Rem


In [ ]:
# ----------------------------------
# 9. Merging and Saving the Model (New Section)
# ----------------------------------

print("Merging LoRA adapters into base model...")
merged_model = model.merge_and_unload()

# Define the output directory to save the merged model
output_dir = "/content/drive/MyDrive/LLM Model/mistral_merged_rag_model"

# Save the merged model and tokenizer
# This will create a single, new model file on your drive
print(f"Saving merged model to {output_dir}")
merged_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Merged model with fine-tuned features saved successfully!")

Merging LoRA adapters into base model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Saving merged model to /content/drive/MyDrive/LLM Model/mistral_merged_rag_model
✅ Merged model with fine-tuned features saved successfully!


In [ ]:
# After restarting the runtime, run this cell to install necessary libraries
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


#Final Code Run to Check the AI Model

In [ ]:
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel # Import PeftModel
from google.colab import userdata # Import the userdata module
import warnings
import json
import pickle
import os

# Suppress a specific UserWarning from SentencePiece that is not critical
warnings.filterwarnings("ignore", category=UserWarning, message="A new version of SentencePiece.*")

class DiabetesChatbot:
    def __init__(self, base_model_name: str, adapter_path: str, faiss_index_path: str, meta_path: str, embedding_model_name: str = 'all-MiniLM-L6-v2'):
        print("Initializing the chatbot...")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")

        # --- Get the Hugging Face token from Colab Secrets ---
        try:
            self.hf_token = userdata.get('HUGGING_FACE_TOKEN')
            if self.hf_token is None:
                print("Warning: HF_TOKEN secret not found. Access to some models might be restricted.")
        except Exception as e:
            print(f"Error accessing Colab secret HF_TOKEN: {e}")
            print("Access to some models might be restricted.")
            self.hf_token = None

        self._load_model(base_model_name, adapter_path)
        self._load_rag_components(faiss_index_path, meta_path, embedding_model_name)
        print("✅ Chatbot initialized and ready.")

    def _load_model(self, base_model_name: str, adapter_path: str):
        print("Loading base model and tokenizer...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        self.tokenizer = AutoTokenizer.from_pretrained(
            base_model_name,
            token=self.hf_token
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
            device_map={'': self.device},
            token=self.hf_token
        )

        print(f"Applying the LoRA adapter from: {adapter_path}")
        self.model = PeftModel.from_pretrained(base_model, adapter_path)
        # Optional: Merge and unload for inference
        # print("Merging LoRA adapters...")
        # self.model = self.model.merge_and_unload()
        self.model.eval() # Set model to evaluation mode


    def _load_rag_components(self, faiss_index_path: str, meta_path: str, embedding_model_name: str):
        print("Loading RAG components (FAISS index, chunks, embedding model)...")
        # Load FAISS index
        if not os.path.exists(faiss_index_path):
             print(f"❌ Error: FAISS index not found at {faiss_index_path}")
             self.index = None
        else:
            self.index = faiss.read_index(faiss_index_path)
            print("✅ FAISS index loaded.")

        # Load metadata (chunks)
        if not os.path.exists(meta_path):
            print(f"❌ Error: Metadata file not found at {meta_path}")
            self.chunks = []
        else:
            with open(meta_path, "rb") as f:
                meta = pickle.load(f)
            self.chunks = meta.get("docs", [])
            # self.metas = meta.get("records", []) # Uncomment if needed for showing sources
            print(f"✅ Loaded {len(self.chunks)} chunks from knowledge base metadata.")


        # Load embedding model
        from sentence_transformers import SentenceTransformer
        self.embedding_model = SentenceTransformer(embedding_model_name)
        print("✅ Embedding model loaded.")


    def get_recommendation_from_doc(self, user_query: str, top_k=3) -> str:
        if not self.index or not self.chunks or not self.embedding_model:
             return "Error: RAG components not loaded. Cannot provide recommendations."

        # Encode query
        query_embedding = self.embedding_model.encode([user_query])
        query_embedding_np = np.array(query_embedding).astype('float32')

        # Search FAISS
        distances, indices = self.index.search(query_embedding_np, top_k)
        relevant_chunks = [self.chunks[i] for i in indices[0]]
        context = " ".join(relevant_chunks)

        # Prompt for RAG
        prompt = f"""
        [INST]
        You are a medical assistant.
        Based on the following diabetes guidelines, answer the user's question clearly and concisely.

        Guidelines: {context}

        Question: {user_query}
        [/INST]
        """

        # Generate answer
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=350, # Increased tokens for more detailed RAG answers
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=self.tokenizer.eos_token_id
            )
        response_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response_text.split("[/INST]")[-1].strip()


    def personalized_recommendation(self, user_profile: dict, query: str, top_k=3) -> str:
        if not self.index or not self.chunks or not self.embedding_model:
             return "Error: RAG components not loaded. Cannot provide personalized recommendations."

        # User profile inject karo
        profile_text = f"Patient Info: Age {user_profile.get('age', 'N/A')}, Gender {user_profile.get('gender', 'N/A')}, Diabetes Type {user_profile.get('type', 'N/A')}, Activity Level {user_profile.get('activity', 'N/A')}."

        # Query embedding
        query_embedding = self.embedding_model.encode([query])
        query_embedding_np = np.array(query_embedding).astype('float32')

        # FAISS search
        distances, indices = self.index.search(query_embedding_np, top_k)
        relevant_chunks = [self.chunks[i] for i in indices[0]]
        context = " ".join(relevant_chunks)

        # Prompt for personalized RAG
        prompt = f"""
        [INST]
        You are a diabetes health assistant.
        Use the patient profile and guidelines to generate a personalized recommendation.

        {profile_text}

        Guidelines: {context}

        Question: {query}
        [/INST]
        """

        # Model response
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=150,  # Adjust tokens as needed
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=self.tokenizer.eos_token_id
            )
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response.split("[/INST]")[-1].strip()


# --- Main execution block ---
if __name__ == "__main__":
    # Define paths to your model and RAG components
    BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
    ADAPTER = "/content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2"
    FAISS_INDEX = "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin"
    META_DATA = "/content/drive/MyDrive/LLM Model/med_index_meta.pkl" # Path to your metadata file

    # Initialize the chatbot
    chatbot = DiabetesChatbot(
        base_model_name=BASE_MODEL,
        adapter_path=ADAPTER,
        faiss_index_path=FAISS_INDEX,
        meta_path=META_DATA
    )

    try:
        if hasattr(chatbot, 'model'): # Check if initialization was successful
            print("\n--- Enter your questions or commands below. Type 'quit' or 'exit' to stop. ---")
            print("Commands: 'profile'")

            current_user_profile = {} # Store user profile data

            while True:
                user_input = input("You: ")

                if user_input.lower() in ["quit", "exit"]:
                    print("Chatbot session ended. Goodbye!")
                    break

                elif user_input.lower() == "profile":
                    print("\n--- Enter Patient Profile ---")
                    current_user_profile['age'] = int(input("Enter Age: "))
                    current_user_profile['gender'] = input("Enter Gender (Male/Female): ")
                    current_user_profile['type'] = input("Enter Diabetes Type (Type 1/Type 2/Gestational/None/Prediabetes): ")
                    current_user_profile['activity'] = input("Enter Activity Level (Low/Moderately active/Very active): ")
                    print("✅ Profile updated!")
                    print(f"Current Profile: {current_user_profile}")
                else:
                    # Use the RAG function for general questions or personalized based on profile
                    print("Generating response...")
                    if current_user_profile:
                         # Use personalized RAG if profile is set
                         response = chatbot.personalized_recommendation(current_user_profile, user_input)
                    else:
                         # Use general RAG if no profile
                         response = chatbot.get_recommendation_from_doc(user_input)

                    print(f"\nChatbot: {response}\n")

    except Exception as e:
        print(f"An error occurred during execution: {e}")

Initializing the chatbot...
Using device: cuda
Loading base model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded.
✅ Chatbot initialized and ready.

--- Enter your questions or commands below. Type 'quit' or 'exit' to stop. ---
Commands: 'profile'
You: iam preganant
Generating response...

Chatbot: Answer: Please consult your healthcare provider for guidance on diabetes management during pregnancy.

You: he is sitting beside me
Generating response...

Chatbot: Answer: The guidelines do not address your question directly, but they do suggest that a patient with tingling in hands/feet should be evaluated for peripheral neuropathy, and treatment with a medication such as Sitagliptin may be recommended to help improve blood glucose control.

You: he is tickling me
Generating response...

Chatbot: Based on the given guidelines, it is unclear whether the user is referring to a symptom or an actual situation. If this is a symptom, it is important to note that tingling in hands and feet is a common symptom of neuropathy, a complication of diabetes. If this is an actual situation, 